In [1]:
from skimage import data, draw,color, data, restoration
from skimage import transform, util
from scipy import misc, fftpack, signal
import numpy as np
from skimage import filters, color
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
import matplotlib.animation as animation
from scipy.misc import imresize
from scipy import ndimage as nd
import scipy
#deixar la linea comentada perque les imatges s'obrin en una nova finestra i es pugui veure l'animació
#%matplotlib inline

In [2]:
rgb =  plt.imread('countryside.jpg')
img = color.rgb2gray(rgb)
img = util.img_as_float(img)
fig = plt.figure(1) 
plt.imshow(rgb.astype('uint8'))
plt.title('Original Image')
plt.show()


In [3]:
def gradient(img):
    img = color.rgb2gray(img)
    gx, gy = np.gradient(img)
    grad = np.sqrt(gx*gx + gy*gy)
    return grad
grad = gradient(img)

fig = plt.figure(2) 
plt.title('Contorns imatge')
plt.imshow(grad, cmap="gray" )
plt.show()

In [4]:
#normalitzacio d'imatge
def normalitzar(imatge):
    return np.real((imatge - np.min(imatge)) * 256 / (np.max(imatge) - np.min(imatge)))

In [5]:

#Calcul matriu d'energia que acomulada te cada zona
def energia(imatge):
    height,width = imatge.shape
    energia=np.zeros(imatge.shape)
    #asignem la primera linea igual que la imatge que tenim
    energia[0]=imatge[0]   
    for x in range(1,height):
        for y in range(0,width):
            if y == 0:#inici de la imatge
                min_val = min( energia[x-1, y], energia[x-1, y+1] )
            elif y < width - 2:#abans del final de la imatge
                min_val = min( energia[x-1, y], energia[x-1, y+1] )
                min_val = min( min_val, energia[x-1, y-1] )
            else:#final de la imatge
                min_val = min( energia[x-1, y], energia[x-1, y-1] )
            energia[x,y] = imatge[x,y] + min_val#valor acomulat d'energia
    return energia
            
    
    

In [6]:
grad=normalitzar(grad)
E = energia(grad)
fig = plt.figure(3) 
plt.title('Acumulacio energia')
plt.imshow(E, cmap="gray" )
plt.show()




In [7]:
# El calcul del tall vertical hem de buscar el cami amb cost minim. BACKTRACKING
# Aquest tall pot ser vertical o horitzontal, es un camí de pixels connectats. En el nostre cas l'apliquem vertical
# La función de importancia / energía valora un píxel mediante la medición de su contraste con sus píxeles vecinos.
# Des de la imatge en escala de grisos retornarem el tall vertical mes baix com a llista de pixels

def tall_vertical(grad,cost):
    # Anem fins al -1 perque aixi es quedara en el 0, que tambe el necessitem.
    width, height = grad.shape
    min_val =np.max(cost)+1
    path = []
    for y in range(height):
        if cost[width-1,y] < min_val:
            min_val = cost[width-1,y]
            min_ptr = y
            
    pos = (width-1, min_ptr)
    path.append(pos) 
    while pos[0] != 0:
        val = cost[pos] - grad[pos]
        x,y = pos
        if y == 0:
            if val == cost[x-1,y+1]:
                pos = (x-1,y+1)
            else:
                pos = (x-1,y)
        elif y <= height - 2:
            if val == cost[x-1,y+1]:
                pos = (x-1,y+1)
            elif val == cost[x-1,y]:
                pos = (x-1,y)
            else:
                pos = (x-1,y-1)
        else:
            if val == cost[x-1,y]:
                pos = (x-1,y)
            else:
                pos = (x-1,y-1)
                
        path.append(pos)  
    return path 


In [8]:
def borrar_cami (img, path):
    height, width = img.shape[:2]
    img_retallada = np.zeros((height, width-1, img.shape[2]))
    path_cpy = set(path)
    visitat = set()
    #print path[1]
    for x in range(height):
        for y in range(width):
            if x not in visitat and (x,y) not in path_cpy:
                img_retallada[x,y] = img[x,y]
            elif (x,y) in path_cpy:
                visitat.add(x)
            else:
                img_retallada[x,y-1] = img[x,y]
    #print img_retallada.shape
    return img_retallada;

def mark_seam (mat, path, mark_as='red'):
    #Dono maxim valor al canal corresponent al color escollit a tots els pixels del path que ens marca el cami a borrar
    for i in path:
        if mark_as == 'red':
            mat[i] = (255,0,0)
        elif mark_as == 'green':
            mat[i] = (0,255,0)
        elif mark_as == 'blue':
            mat[i] = (0,0,255)
        elif mark_as == 'white':
            mat[i] = (255,255,255)
        elif mark_as == 'black':
            mat[i] = (0,0,0)
    
    return mat;          


In [9]:
def carving(imatge,iterations):
    path = []
    rgb = imatge
    seams = []
    for t in range(iterations): 
        path = []
        grad=gradient(rgb)
        E = energia(grad)
        path = tall_vertical(grad,E)
        rgb = mark_seam(rgb, path, 'red')
        seams.append([plt.imshow(rgb.astype('uint8'))])
        rgb = borrar_cami(rgb, path)
    return rgb,seams;


In [10]:
rgb =  plt.imread('countryside.jpg')
imatge_reduida, seams=carving(rgb,100)
fig = plt.figure(4) 
plt.title('Imatge Retallada')
plt.imshow(imatge_reduida.astype('uint8'))

#l'animació s'actibara en el seguent pas i podrem veure-la si tenim comentada la linea de %matplotlib inline al inici del document

In [11]:
######################Comparacio RESULTATS
ani = animation.ArtistAnimation(fig, seams, interval=100, blit=True, repeat_delay=1000);
plt.show();
scipy.misc.imsave('Reduccion_final.jpg', imatge_reduida)
fig = plt.figure(5)
plt.subplot(1,2,1)
plt.imshow(plt.imread('countryside.jpg').astype('uint8'))
plt.title('Original')
plt.subplot(1,2,2)
plt.imshow(rgb.astype('uint8'))
plt.title('Reduccion 100 iteracions')
plt.gcf().set_size_inches((15,15))
plt.show()

